# Tool Selection Strategies: Improving Tool Choice

## Introduction

In this advanced notebook, you'll learn strategies to improve how LLMs select tools. When you have many tools, the LLM can get confused about which one to use. You'll learn techniques to make tool selection more reliable and accurate.

### What You'll Learn

- Common tool selection failures
- Strategies to improve tool selection
- Clear naming conventions
- Detailed descriptions with examples
- Testing and debugging tool selection

### Prerequisites

- Completed `02_defining_tools.ipynb`
- Redis 8 running locally
- OpenAI API key set
- Course data ingested

## Concepts: Tool Selection Challenges

### The Problem

As you add more tools, the LLM faces challenges:

**With 3 tools:**
- ✅ Easy to choose
- ✅ Clear distinctions

**With 10+ tools:**
- ⚠️ Similar-sounding tools
- ⚠️ Overlapping functionality
- ⚠️ Ambiguous queries
- ⚠️ Wrong tool selection

### Common Tool Selection Failures

**1. Similar Names**
```python
# Bad: Confusing names
get_course()      # Get one course?
get_courses()     # Get multiple courses?
search_course()   # Search for courses?
find_courses()    # Find courses?
```

**2. Vague Descriptions**
```python
# Bad: Too vague
def search_courses():
    """Search for courses."""
    
# Good: Specific
def search_courses():
    """Search for courses using semantic search.
    Use when students ask about topics, departments, or characteristics.
    Example: 'machine learning courses' or 'online courses'
    """
```

**3. Overlapping Functionality**
```python
# Bad: Unclear when to use which
search_courses(query)           # Semantic search
filter_courses(department)      # Filter by department
find_courses_by_topic(topic)    # Find by topic

# Good: One tool with clear parameters
search_courses(query, filters)  # One tool, clear purpose
```

### How LLMs Select Tools

The LLM considers:
1. **Tool name** - First impression
2. **Tool description** - Main decision factor
3. **Parameter descriptions** - Confirms choice
4. **Context** - User's query and conversation

**Key insight:** The LLM can't see your code, only the schema!

## Setup

In [ ]:
import os
from typing import List, Optional, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from redis_context_course import CourseManager

# Initialize
llm = ChatOpenAI(model="gpt-4o", temperature=0)
course_manager = CourseManager()

print("✅ Setup complete")

## Strategy 1: Clear Naming Conventions

Use consistent, descriptive names that clearly indicate what the tool does.

### Bad Example: Confusing Names

In [ ]:
# Bad: Confusing, similar names
class GetCourseInput(BaseModel):
    code: str = Field(description="Course code")

@tool(args_schema=GetCourseInput)
async def get(code: str) -> str:
    """Get a course."""
    course = await course_manager.get_course(code)
    return str(course) if course else "Not found"

@tool(args_schema=GetCourseInput)
async def fetch(code: str) -> str:
    """Fetch a course."""
    course = await course_manager.get_course(code)
    return str(course) if course else "Not found"

@tool(args_schema=GetCourseInput)
async def retrieve(code: str) -> str:
    """Retrieve a course."""
    course = await course_manager.get_course(code)
    return str(course) if course else "Not found"

print("❌ BAD: Three tools that do the same thing with vague names!")
print("   - get, fetch, retrieve - which one to use?")
print("   - LLM will be confused")

### Good Example: Clear, Descriptive Names

In [ ]:
# Good: Clear, specific names
class SearchCoursesInput(BaseModel):
    query: str = Field(description="Natural language search query")
    limit: int = Field(default=5, description="Max results")

@tool(args_schema=SearchCoursesInput)
async def search_courses_by_topic(query: str, limit: int = 5) -> str:
    """Search courses using semantic search based on topics or descriptions."""
    results = await course_manager.search_courses(query, limit=limit)
    return "\n".join([f"{c.course_code}: {c.title}" for c in results])

class GetCourseDetailsInput(BaseModel):
    course_code: str = Field(description="Specific course code like 'CS101'")

@tool(args_schema=GetCourseDetailsInput)
async def get_course_details_by_code(course_code: str) -> str:
    """Get detailed information about a specific course by its course code."""
    course = await course_manager.get_course(course_code)
    return str(course) if course else "Course not found"

class ListCoursesInput(BaseModel):
    department: str = Field(description="Department code like 'CS' or 'MATH'")

@tool(args_schema=ListCoursesInput)
async def list_courses_by_department(department: str) -> str:
    """List all courses in a specific department."""
    # Implementation would filter by department
    return f"Courses in {department} department"

print("✅ GOOD: Clear, specific names that indicate purpose")
print("   - search_courses_by_topic: For semantic search")
print("   - get_course_details_by_code: For specific course")
print("   - list_courses_by_department: For department listing")

## Strategy 2: Detailed Descriptions with Examples

Write descriptions that explain WHEN to use the tool, not just WHAT it does.

### Bad Example: Vague Description

In [ ]:
# Bad: Vague description
@tool(args_schema=SearchCoursesInput)
async def search_courses_bad(query: str, limit: int = 5) -> str:
    """Search for courses."""
    results = await course_manager.search_courses(query, limit=limit)
    return "\n".join([f"{c.course_code}: {c.title}" for c in results])

print("❌ BAD: 'Search for courses' - too vague!")
print("   - When should I use this?")
print("   - What kind of search?")
print("   - What queries work?")

### Good Example: Detailed Description with Examples

In [ ]:
# Good: Detailed description with examples
@tool(args_schema=SearchCoursesInput)
async def search_courses_good(query: str, limit: int = 5) -> str:
    """
    Search for courses using semantic search based on topics, descriptions, or characteristics.
    
    Use this tool when students ask about:
    - Topics or subjects: "machine learning courses", "database courses"
    - Course characteristics: "online courses", "beginner courses", "3-credit courses"
    - General exploration: "what courses are available in AI?"
    
    Do NOT use this tool when:
    - Student asks about a specific course code (use get_course_details_by_code instead)
    - Student wants all courses in a department (use list_courses_by_department instead)
    
    The search uses semantic matching, so natural language queries work well.
    
    Examples:
    - "machine learning courses" → finds CS401, CS402, etc.
    - "beginner programming" → finds CS101, CS102, etc.
    - "online data science courses" → finds online courses about data science
    """
    results = await course_manager.search_courses(query, limit=limit)
    return "\n".join([f"{c.course_code}: {c.title}" for c in results])

print("✅ GOOD: Detailed description with:")
print("   - What it does")
print("   - When to use it")
print("   - When NOT to use it")
print("   - Examples of good queries")

## Strategy 3: Parameter Descriptions

Add detailed descriptions to parameters to guide the LLM.

In [ ]:
# Bad: Minimal parameter descriptions
class BadInput(BaseModel):
    query: str
    limit: int

print("❌ BAD: No parameter descriptions")
print()

# Good: Detailed parameter descriptions
class GoodInput(BaseModel):
    query: str = Field(
        description="Natural language search query. Can be topics (e.g., 'machine learning'), "
                    "characteristics (e.g., 'online courses'), or general questions "
                    "(e.g., 'beginner programming courses')"
    )
    limit: int = Field(
        default=5,
        description="Maximum number of results to return. Default is 5. "
                    "Use 3 for quick answers, 10 for comprehensive results."
    )

print("✅ GOOD: Detailed parameter descriptions")
print("   - Explains what the parameter is")
print("   - Gives examples")
print("   - Suggests values")

## Testing Tool Selection

Let's test how well the LLM selects tools with different queries.

In [ ]:
# Create tools with good descriptions
tools = [
    search_courses_good,
    get_course_details_by_code,
    list_courses_by_department
]

llm_with_tools = llm.bind_tools(tools)

# Test queries
test_queries = [
    "I'm interested in machine learning courses",
    "Tell me about CS401",
    "What courses does the Computer Science department offer?",
    "Show me beginner programming courses",
    "What are the prerequisites for CS301?",
]

print("=" * 80)
print("TESTING TOOL SELECTION")
print("=" * 80)

for query in test_queries:
    messages = [
        SystemMessage(content="You are a class scheduling agent. Use the appropriate tool."),
        HumanMessage(content=query)
    ]
    
    response = llm_with_tools.invoke(messages)
    
    print(f"\nQuery: {query}")
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        print(f"✅ Selected: {tool_call['name']}")
        print(f"   Args: {tool_call['args']}")
    else:
        print("❌ No tool selected")

print("\n" + "=" * 80)

## Strategy 4: Testing Edge Cases

Test ambiguous queries to find tool selection issues.

In [ ]:
# Ambiguous queries that could match multiple tools
ambiguous_queries = [
    "What courses are available?",  # Could be search or list
    "Tell me about CS courses",     # Could be search or list
    "I want to learn programming",  # Could be search
    "CS401",                        # Just a course code
]

print("=" * 80)
print("TESTING AMBIGUOUS QUERIES")
print("=" * 80)

for query in ambiguous_queries:
    messages = [
        SystemMessage(content="You are a class scheduling agent. Use the appropriate tool."),
        HumanMessage(content=query)
    ]
    
    response = llm_with_tools.invoke(messages)
    
    print(f"\nQuery: '{query}'")
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        print(f"Selected: {tool_call['name']}")
        print(f"Args: {tool_call['args']}")
        print("Is this the right choice? 🤔")
    else:
        print("No tool selected - might ask for clarification")

print("\n" + "=" * 80)
print("💡 TIP: If selection is wrong, improve tool descriptions!")

## Strategy 5: Reducing Tool Confusion

When you have many similar tools, consider consolidating them.

In [ ]:
print("=" * 80)
print("CONSOLIDATING SIMILAR TOOLS")
print("=" * 80)

print("\n❌ BAD: Many similar tools")
print("   - search_courses_by_topic()")
print("   - search_courses_by_department()")
print("   - search_courses_by_difficulty()")
print("   - search_courses_by_format()")
print("   → LLM confused about which to use!")

print("\n✅ GOOD: One flexible tool")
print("   - search_courses(query, filters={})")
print("   → One tool, clear purpose, flexible parameters")

# Example of consolidated tool
class ConsolidatedSearchInput(BaseModel):
    query: str = Field(description="Natural language search query")
    department: Optional[str] = Field(default=None, description="Filter by department (e.g., 'CS')")
    difficulty: Optional[str] = Field(default=None, description="Filter by difficulty (beginner/intermediate/advanced)")
    format: Optional[str] = Field(default=None, description="Filter by format (online/in-person/hybrid)")
    limit: int = Field(default=5, description="Max results")

@tool(args_schema=ConsolidatedSearchInput)
async def search_courses_consolidated(
    query: str,
    department: Optional[str] = None,
    difficulty: Optional[str] = None,
    format: Optional[str] = None,
    limit: int = 5
) -> str:
    """
    Search for courses with optional filters.
    
    Use this tool for any course search. You can:
    - Search by topic: query="machine learning"
    - Filter by department: department="CS"
    - Filter by difficulty: difficulty="beginner"
    - Filter by format: format="online"
    - Combine filters: query="databases", department="CS", difficulty="intermediate"
    """
    # Implementation would use filters
    return f"Searching for: {query} with filters"

print("\n✅ Benefits of consolidation:")
print("   - Fewer tools = less confusion")
print("   - One clear purpose")
print("   - Flexible with optional parameters")

## Key Takeaways

### Naming Conventions

✅ **Do:**
- Use descriptive, action-oriented names
- Include the object/entity in the name
- Be specific: `search_courses_by_topic` not `search`

❌ **Don't:**
- Use vague names: `get`, `fetch`, `find`
- Create similar-sounding tools
- Use abbreviations or jargon

### Description Best Practices

Include:
1. **What it does** - Clear explanation
2. **When to use it** - Specific scenarios
3. **When NOT to use it** - Avoid confusion
4. **Examples** - Show expected inputs
5. **Edge cases** - Handle ambiguity

### Parameter Descriptions

For each parameter:
- Explain what it is
- Give examples
- Suggest typical values
- Explain constraints

### Testing Strategy

1. **Test typical queries** - Does it select correctly?
2. **Test edge cases** - What about ambiguous queries?
3. **Test similar queries** - Does it distinguish between tools?
4. **Iterate descriptions** - Improve based on failures

### When to Consolidate Tools

Consolidate when:
- ✅ Tools have similar purposes
- ✅ Differences can be parameters
- ✅ LLM gets confused

Keep separate when:
- ✅ Fundamentally different operations
- ✅ Different return types
- ✅ Clear, distinct use cases

## Exercises

1. **Improve a tool**: Take a tool with a vague description and rewrite it with examples and clear guidance.

2. **Test tool selection**: Create 10 test queries and verify the LLM selects the right tool each time.

3. **Find confusion**: Create two similar tools and test queries that could match either. How can you improve the descriptions?

4. **Consolidate tools**: If you have 5+ similar tools, try consolidating them into 1-2 flexible tools.

## Summary

In this notebook, you learned:

- ✅ Clear naming conventions prevent confusion
- ✅ Detailed descriptions with examples guide tool selection
- ✅ Parameter descriptions help the LLM use tools correctly
- ✅ Testing edge cases reveals selection issues
- ✅ Consolidating similar tools reduces confusion

**Key insight:** Tool selection quality depends entirely on your descriptions. The LLM can't see your code - invest time in writing clear, detailed tool schemas with examples and guidance.